In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import requests                   # Uses the requests library for REST apis
import os                         # Loads operating system libraries
from ldaca.ldaca import LDaCA     # Loads the LDaCA ReST api wrapper
from rocrate_lang.utils import as_list # A handy utility for converting to list

In [ ]:
print("hi")